In [24]:
import os
import numpy as np
from io import BytesIO
import tempfile
from glob import glob

#%matplotlib notebook
import matplotlib.pyplot as plt
import pandas as pd
import sqlite3
import lsst.daf.persistence as dafPersist
import lsst.afw.display as afwDisplay
import lsst.afw.image as afwImage
import lsst.afw.fits as afwFits
import lsst.geom
import lsst.alert.packet
afwDisplay.setDefaultBackend('matplotlib')
import matplotlib.ticker as plticker
from astropy.visualization import (ZScaleInterval, SqrtStretch, ImageNormalize)
from astropy.utils.console import ProgressBar
from astropy.nddata import CCDData

import fastavro

In [45]:
#fastavro.schema._schema.SCHEMA_DEFS.clear()

In [4]:
!ls /project/morriscb/src/ap_verify_hits2015/DM-25930/alerts | head

41091501.avro
41091503.avro
41091504.avro
41091505.avro
41091506.avro
41091507.avro
41091508.avro
41091509.avro
41091510.avro
41091511.avro
ls: write error: Broken pipe


In [5]:
alert_files = glob('/project/morriscb/src/ap_verify_hits2015/DM-25930/alerts/*.avro')

In [6]:
len(alert_files)

5020

In [11]:
avro_file = alert_files[2]

In [44]:
with open(f'{avro_file}','rb') as f:
    ct = 0
    writer_schema, packet_iter = schema.retrieve_alerts(f)
    for packet in packet_iter:
        ct += 1
print(ct)

SchemaResolutionError: Schema mismatch: {'type': 'array', 'items': {'type': 'record', 'name': 'lsst.v3_0.diaForcedSource', 'fields': [{'name': 'diaForcedSourceId', 'type': 'long'}, {'name': 'ccdVisitId', 'type': 'long'}, {'name': 'diaObjectId', 'type': 'long'}, {'name': 'midPointTai', 'type': 'double'}, {'name': 'filterName', 'type': 'string'}, {'name': 'psFlux', 'type': 'float'}, {'name': 'psFluxErr', 'type': 'float'}, {'name': 'totFlux', 'type': 'float'}, {'name': 'totFluxErr', 'type': 'float'}]}} is not null

In [14]:
schema.definition

{'type': 'record',
 'name': 'lsst.alert',
 'fields': [{'doc': 'unique alert identifer',
   'name': 'alertId',
   'type': 'long'},
  {'name': 'diaSource',
   'type': {'type': 'record',
    'name': 'lsst.diaSource',
    'fields': [{'name': 'diaSourceId', 'type': 'long'},
     {'name': 'ccdVisitId', 'type': 'long'},
     {'default': None, 'name': 'diaObjectId', 'type': ['null', 'long']},
     {'default': None, 'name': 'ssObjectId', 'type': ['null', 'long']},
     {'default': None, 'name': 'parentDiaSourceId', 'type': ['null', 'long']},
     {'name': 'midPointTai', 'type': 'double'},
     {'name': 'filterName', 'type': 'string'},
     {'name': 'programId', 'type': 'int'},
     {'name': 'ra', 'type': 'double'},
     {'name': 'decl', 'type': 'double'},
     {'default': None, 'name': 'raErr', 'type': ['null', 'float']},
     {'default': None, 'name': 'declErr', 'type': ['null', 'float']},
     {'default': None, 'name': 'ra_decl_Cov', 'type': ['null', 'float']},
     {'name': 'x', 'type': 'flo

In [50]:
f = open(avro_file,'rb')
freader = fastavro.reader(f)
writer_schema = freader.writer_schema


In [38]:
writer_schema

{'type': 'record',
 'name': 'lsst.v3_0.alert',
 'fields': [{'doc': 'unique alert identifer',
   'name': 'alertId',
   'type': 'long'},
  {'name': 'diaSource',
   'type': {'type': 'record',
    'name': 'lsst.v3_0.diaSource',
    'fields': [{'name': 'diaSourceId', 'type': 'long'},
     {'name': 'ccdVisitId', 'type': 'long'},
     {'default': None, 'name': 'diaObjectId', 'type': ['null', 'long']},
     {'default': None, 'name': 'ssObjectId', 'type': ['null', 'long']},
     {'default': None, 'name': 'parentDiaSourceId', 'type': ['null', 'long']},
     {'name': 'midPointTai', 'type': 'double'},
     {'name': 'filterName', 'type': 'string'},
     {'name': 'programId', 'type': 'int'},
     {'name': 'ra', 'type': 'double'},
     {'name': 'decl', 'type': 'double'},
     {'default': None, 'name': 'raErr', 'type': ['null', 'float']},
     {'default': None, 'name': 'declErr', 'type': ['null', 'float']},
     {'default': None, 'name': 'ra_decl_Cov', 'type': ['null', 'float']},
     {'name': 'x', 't

In [51]:
ct = 0
for packet in freader:
    print(packet['prvDiaForcedSources'])
    ct += 1
print(ct)

[{'diaForcedSourceId': 88492288543556544, 'ccdVisitId': 41207433, 'diaObjectId': 176984577087111203, 'midPointTai': 57073.08373942709, 'filterName': 'g', 'psFlux': nan, 'psFluxErr': nan, 'totFlux': 12293.8134765625, 'totFluxErr': 2046.176025390625}]
[{'diaForcedSourceId': 88492288543556545, 'ccdVisitId': 41207433, 'diaObjectId': 176984577087111204, 'midPointTai': 57073.08373942709, 'filterName': 'g', 'psFlux': nan, 'psFluxErr': nan, 'totFlux': 13651.1689453125, 'totFluxErr': 2188.08984375}]
[{'diaForcedSourceId': 88492288543555956, 'ccdVisitId': 41207433, 'diaObjectId': 177261172980973722, 'midPointTai': 57073.08373942709, 'filterName': 'g', 'psFlux': 25055.69140625, 'psFluxErr': 4077.906005859375, 'totFlux': 1502233.625, 'totFluxErr': 4163.2099609375}, {'diaForcedSourceId': 88319416109891665, 'ccdVisitId': 41126933, 'diaObjectId': 177261172980973722, 'midPointTai': 57071.089641538405, 'filterName': 'g', 'psFlux': 33807.2890625, 'psFluxErr': 1413.9140625, 'totFlux': 1263609.375, 'totFl